# Import data

In [ ]:
import pickle as pkl
import numpy as np

with open('X_train_norm.pkl', 'rb') as f:
    X_train_norm = pkl.load(f)
with open('X_test_norm.pkl', 'rb') as f:
    X_test_norm = pkl.load(f)
with open('X_val_norm.pkl', 'rb') as f:
    X_val_norm = pkl.load(f)
with open('abnorm_beats.pkl', 'rb') as f:
    abnorm_beats = pkl.load(f)

# Autoencoder Architecture

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Activation, TimeDistributed, Dense
from tensorflow.keras.initializers import GlorotUniform
SEED=73
def build_encoder(input_shape, hidden_size1, hidden_size2, hidden_size3, dropout_rate): 
    model = Sequential()
    model.add(LSTM(hidden_size1, return_sequences=True, input_shape=input_shape, kernel_initializer=GlorotUniform(seed=SEED)))
    model.add(Activation('relu'))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(hidden_size2, return_sequences=True, kernel_initializer=GlorotUniform(seed=SEED)))
    model.add(Activation('relu'))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(hidden_size3, return_sequences=True, kernel_initializer=GlorotUniform(seed=SEED)))
    #model.add(Activation('relu'))
    #model.add(Dropout(dropout_rate))
    #model.add(LSTM(hidden_size4, return_sequences=True))
    return model

def build_decoder(input_shape, hidden_size1, hidden_size2, hidden_size3, dropout_rate):
    model = Sequential()
    model.add(LSTM(hidden_size1, return_sequences=True, input_shape=input_shape, kernel_initializer=GlorotUniform(seed=SEED)))
    model.add(Activation('relu'))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(hidden_size2, return_sequences=True, kernel_initializer=GlorotUniform(seed=SEED)))
    model.add(Activation('relu'))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(hidden_size3, return_sequences=True, kernel_initializer=GlorotUniform(seed=SEED)))
    #model.add(Activation('relu'))
    #model.add(Dropout(dropout_rate))
    #model.add(LSTM(hidden_size4, return_sequences=True))
    model.add(TimeDistributed(Dense(1)))
    return model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.optimizers import Adam

window_size = 100
# Define the input shape
input_shape = (window_size, 1)

# Define the size of the hidden layers
hidden_size1 = 128
hidden_size2 = 64
hidden_size3 = 32
hidden_size4=16
dropout_rate = 0.2

# Build the encoder
encoder_input = Input(shape=input_shape)
encoder_output = build_encoder(input_shape, hidden_size1, hidden_size2, hidden_size3, dropout_rate)(encoder_input)
# Build the decoder
decoder_output = build_decoder((window_size, hidden_size3), hidden_size3, hidden_size2, hidden_size1, dropout_rate)(encoder_output)

# Connect the encoder and decoder into an autoencoder
autoencoder = Model(encoder_input, decoder_output, name='Autoencoder')
# Compile the model
autoencoder.compile(loss=MeanAbsoluteError(), optimizer=Adam(),
                    metrics=[MeanAbsoluteError(),MeanSquaredError()])

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
# create callbacks
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.9,
    patience=5,
    verbose=0,
    mode='min',
    min_delta=1e-7,
    min_lr=1e-6,
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    mode='min',
    min_delta=1e-7,
    restore_best_weights=True,
)

callbacks = [lr_scheduler, early_stopping]

epochs = 1000
batch_size = 1024

# Train the autoencoder model
autoencoder_history = autoencoder.fit(X_train_norm, X_train_norm,
                                      epochs=epochs, batch_size=batch_size,
                                      validation_data=(X_val_norm, X_val_norm),
                                      callbacks=callbacks
                                      ).history

In [ ]:
import matplotlib.pyplot as plt
def plot_learning_curves(model_history):
    """
    Plot learning curves for accuracy, loss, and learning rate.

    Parameters:
        model_history (tf.keras.callbacks.History): History object obtained during model training.
    """
    best_epoch = np.argmin(model_history['val_loss'])
    # show accuracy curve
    plt.figure(figsize=(20,5))
    plt.plot(model_history['mean_absolute_error'], label='Mean Absolute Error [train]', alpha=.8, color='#ff7f0e')
    plt.plot(model_history['val_mean_absolute_error'], label='Mean Absolute Error [val]', alpha=.9, color='#5a9aa5')
    plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
    plt.title('Mean Absolute Error')
    plt.xlabel('epoch')
    plt.ylabel('Mean Absolute Error')
    plt.legend()
    plt.grid(alpha=.3)
    # show loss curve
    plt.figure(figsize=(20,5))
    plt.plot(model_history['mean_squared_error'], label='Mean Squared Error Loss', alpha=.8, color='#ff7f0e')
    plt.plot(model_history['val_mean_squared_error'], label='Mean Squared Error Loss', alpha=.9, color='#5a9aa5')
    plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
    plt.title('Loss')
    plt.xlabel('epoch')
    plt.ylabel('Mean Squared Error Loss')
    plt.legend()
    plt.grid(alpha=.3)
    # show learning rate curve
    plt.figure(figsize=(18,3))
    plt.plot(model_history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
    plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
    plt.title('Learning Rate')
    plt.xlabel('epoch')
    plt.ylabel('Learning rate')
    plt.legend()
    plt.grid(alpha=.3)

    plt.show()

In [ ]:
# Visualize the learning curves
plot_learning_curves(autoencoder_history)

In [ ]:
#balance test set
X_test_norm_1= X_test_norm[0:len(abnorm_beats)]

In [ ]:
# Perform predictions for test set and abnormal set
reconstruction_norm = autoencoder.predict(X_test_norm_1)
reconstruction_abnormal = autoencoder.predict(abnorm_beats)

In [ ]:
# Compute reconstruction losses
mse_norm = np.mean(np.power(X_test_norm_1 - reconstruction_norm, 2), axis=1)
mse_abnormal = np.mean(np.power(abnorm_beats - reconstruction_abnormal, 2), axis=1)

mae_norm = np.mean(np.abs(X_test_norm_1 - reconstruction_norm), axis=1)
mae_abnormal = np.mean(np.abs(abnorm_beats - reconstruction_abnormal), axis=1)

plot for MAE

In [ ]:
import seaborn as sns
mae=1
mse=0
# Plot the histograms of the reconstruction losses
plt.figure(figsize=(30, 6))
if(mse):
  sns.distplot(mse_norm, bins=50, kde=True, label='normal')
  sns.distplot(mse_abnormal, bins=50, kde=True, label='abnormal')
elif(mae):
  sns.distplot(mae_norm, bins=50, kde=True, label='normal')
  sns.distplot(mae_abnormal, bins=50, kde=True, label='abnormal')
xlim=np.arange(0,0.05,0.001)
plt.xlim([0, 0.03])
plt.xticks(xlim)
plt.xlabel('Reconstruction loss')
plt.ylabel('Frequency')
plt.legend(loc='upper right')
plt.show()

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def plot_conf_matrix(TP, TN, FP, FN):
    # Create the confusion matrix
    conf_matrix = np.array([[TN, FP],
                            [FN, TP]])

    # Display the confusion matrix using seaborn heatmap
    labels = ['Normal', 'Abnormal']
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def plot_conf_matrix(TP, TN, FP, FN, normalize=False):
    # Create the confusion matrix
    conf_matrix = np.array([[TN, FP],
                            [FN, TP]])

    # Normalize the confusion matrix if requested
    if normalize:
        conf_matrix = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]

    # Display the confusion matrix using seaborn heatmap
    labels = ['Normal', 'Abnormal']
    sns.heatmap(conf_matrix, annot=True, fmt='.2f' if normalize else 'd', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()




# MAE Analysis

In [ ]:
TN=0 #TN normal
TP=0 #abnormal
FP=0#normal classified as abnormal
FN=0 #abnormal classified as normal
TPR=[]
FPR=[]
vect = np.arange(0, 1, 0.001)
for threshold in vect:
    TN=0 #TN normal
    TP=0 #abnormal
    FP=0#normal classified as abnormal
    FN=0 
    for i in mae_norm:
        if i<= threshold:
            TN= TN+1
        else: FP=FP+1
    for i in mae_abnormal:
        if i<= threshold:
            FN=FN+1
        else: TP=TP+1
    tpr=TP/(TP+FN)
    fpr=FP/(FP+TN)
    TPR.append(tpr)
    FPR.append(fpr)

In [ ]:
from sklearn.metrics import roc_curve, auc
roc_auc = auc(FPR, TPR)
plt.figure(figsize=(8, 6))
plt.plot(FPR, TPR, color='darkorange', lw=2, label='ROC curve (AUC = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Guess')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve MAE')
plt.legend(loc='lower right')
plt.show()

In [ ]:
from sklearn.metrics import recall_score

def custom_metric(recall_normal, recall):
    
    
    # Define the minimum recall for 'N' beats
    min_recall = 0.9
    
    # If the recall for 'N' beats is below the threshold, return penalized recall for abnormal beats
    if recall_normal < min_recall:
        penalty = recall_normal
        return round(recall*penalty,2)
    
    # Otherwise, return the recall for abnormal beats
    return recall


**method 1: maximise TPR-FPR**

In [ ]:
vect = np.arange(0, 1, 0.001)
TPR=[]
FPR=[]
for threshold in vect:
    TN = 0
    TP = 0
    FP = 0
    FN = 0
    for i in mae_norm:
        if i<= threshold:
            TN= TN+1
        else: FP=FP+1
    for i in mae_abnormal:
        if i<= threshold:
            FN=FN+1
        else: TP=TP+1

    tpr = TP / (TP + FN)
    fpr = FP / (FP + TN)
    TPR.append(tpr)
    FPR.append(fpr)


# Find the best threshold
best_threshold_index = np.argmax(np.array(TPR) - np.array(FPR))
best_threshold1 = vect[best_threshold_index]


In [ ]:
def param_with_best_threshold(mae_norm,best_threshold1):
    TN = 0
    TP = 0
    FP = 0
    FN = 0

    for i in mae_norm:
        if i <= best_threshold1:
            TN += 1
        else:
            FP += 1

    for i in mae_abnormal:
        if i <= best_threshold1:
            FN += 1
        else:
            TP += 1

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    return TP, TN,FP,FN, precision, recall


In [ ]:
TP, TN,FP,FN, precision, recall= param_with_best_threshold(mae_norm,best_threshold1)

In [ ]:
plot_conf_matrix(TP,TN,FP,FN, normalize=True)

In [ ]:
from sklearn.metrics import classification_report

# Assuming TP, TN, FP, FN are defined
y_true = [0] * (TN + FP) + [1] * (TP + FN)
y_pred = [0] * TN + [1] * FP + [0] * FN + [1] * TP

report = classification_report(y_true, y_pred)
print(report)


**method 2: maximise custom metric**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def find_threshold(mae_norm, mae_abnormal):

    best_precision=0
    best_recall=0
    custom=[]
    precision=[]
    recall=[]
    tr_neg=[]
    recall_normal=[]
    
    vect = np.arange(0, 1, 0.001)
    for i,threshold in enumerate(vect):
        TN=0 #TN normal
        TP=0 #abnormal
        FP=0#normal classified as abnormal
        FN=0 
        for i in mae_norm:
            if i<= threshold:
                TN= TN+1
            else: 
                FP=FP+1
        for i in mae_abnormal:
            if i<= threshold:
                FN=FN+1
            else: 
                TP=TP+1
        
        precision.append(TP / (TP + FP)) if (TP + FP) != 0 else 0
        recall.append(TP / (TP + FN)) if (TP + FN) != 0 else 0
        recall_normal.append(TN / (TN + FP)) if (TN + FP) != 0 else 0

        custom.append(custom_metric(recall_normal[-1], recall[-1]))

    # Find the best threshold
    best_threshold_index = np.argmax(custom)
    best_threshold = vect[best_threshold_index]
    best_precision=precision[best_threshold_index]
    best_recall=recall[best_threshold_index]
    best_custom=custom[best_threshold_index]
    return best_threshold, best_precision,best_recall, best_custom


In [ ]:
best_threshold2, best_precision,best_recall, best_custom=find_threshold(mae_norm, mae_abnormal)

In [ ]:
TP2, TN2,FP2,FN2, precision2, recall2=param_with_best_threshold(mae_norm,best_threshold2)

plot_conf_matrix(TP2,TN2,FP2,FN2, normalize=True)

In [ ]:
# Assuming TP, TN, FP, FN are defined
y_true = [0] * (TN2 + FP2) + [1] * (TP2 + FN2)
y_pred = [0] * TN2 + [1] * FP2 + [0] * FN2 + [1] * TP2

report = classification_report(y_true, y_pred)
print(report)

# MSE Analisys

**method 1: maximise TPR-FPR**

In [ ]:
TN=0 #TN normal
TP=0 #abnormal
FP=0#normal classified as abnormal
FN=0 #abnormal classified as normal
TPR=[]
FPR=[]
vect = np.arange(0, 1, 0.001)
for threshold in vect:
    TN=0 #TN normal
    TP=0 #abnormal
    FP=0#normal classified as abnormal
    FN=0 
    for i in mse_norm:
        if i<= threshold:
            TN= TN+1
        else: 
            FP=FP+1
    for i in mse_abnormal:
        if i<= threshold:
            FN=FN+1
        else: 
            TP=TP+1
    tpr=TP/(TP+FN)
    fpr=FP/(FP+TN)
    TPR.append(tpr)
    FPR.append(fpr)

In [ ]:
from sklearn.metrics import roc_curve, auc
roc_auc = auc(FPR, TPR)
plt.figure(figsize=(8, 6))
plt.plot(FPR, TPR, color='darkorange', lw=2, label='ROC curve (AUC = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Guess')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve MAE')
plt.legend(loc='lower right')
plt.show()

In [ ]:
vect = np.arange(0, 1, 0.001)
TPR=[]
FPR=[]
for threshold in vect:
    TN = 0
    TP = 0
    FP = 0
    FN = 0
    for i in mae_norm:
        if i<= threshold:
            TN= TN+1
        else: FP=FP+1
    for i in mae_abnormal:
        if i<= threshold:
            FN=FN+1
        else: TP=TP+1

    tpr = TP / (TP + FN)
    fpr = FP / (FP + TN)
    TPR.append(tpr)
    FPR.append(fpr)


# Find the best threshold
best_threshold_index = np.argmax(np.array(TPR) - np.array(FPR))
best_threshold1_mse = vect[best_threshold_index]

In [ ]:
TP_mse1,TN_mse1,FP_mse1,FN_mse1, precision_mse1, recall_mse1= param_with_best_threshold(mse_norm,best_threshold1_mse)
plot_conf_matrix(TP_mse1,TN_mse1,FP_mse1,FN_mse1, normalize=True)

In [ ]:
# Assuming TP, TN, FP, FN are defined
y_true = [0] * (TN_mse1 + FP_mse1) + [1] * (TP_mse1 + FN_mse1)
y_pred = [0] * TN_mse1 + [1] * FP_mse1 + [0] * FN_mse1 + [1] * TP_mse1

report = classification_report(y_true, y_pred)
print(report)

**method 2: maximise custom metric**

In [ ]:
best_threshold2_mse, best_precision_2mse,best_recall2_mse, best_custom_mse=find_threshold(mse_norm, mse_abnormal)

TP_mse2, TN_mse2,FP_mse2,FN_mse2, precision_mse2, recall_mse2=param_with_best_threshold(mse_norm,best_threshold2_mse)

plot_conf_matrix(TP_mse2, TN_mse2,FP_mse2,FN_mse2, normalize=True)

In [ ]:
y_true = [0] * (TN_mse2 + FP_mse2) + [1] * (TP_mse2 + FN_mse2)
y_pred = [0] * TN_mse2 + [1] * FP_mse2 + [0] * FN_mse2 + [1] * TP_mse2

report = classification_report(y_true, y_pred)
print(report)

In [ ]:
#Defien min and max of MAE
MAE= np.array([mae_norm, mae_abnormal])
max_mae=np.max(MAE)
min_mae=np.min(MAE)


In [ ]:
def confidence(beat, rec_beat, threshold, max_mae, min_mae):
    mae_beat = np.mean(np.abs(beat - rec_beat))
    conf=abs(mae_beat-threshold)
    if mae_beat>=threshold:
        conf_norm=conf/abs(max_mae-threshold)
    else:
        conf_norm=conf/abs(min_mae-threshold)
    return conf_norm

In [ ]:
confid=[]
for j,test in enumerate(X_test_norm_1):
    confid.append(confidence(test, reconstruction_norm[j], best_threshold2, max_mae, min_mae))
for j,test in enumerate(abnorm_beats):
    confid.append(confidence(test, reconstruction_abnormal[j], best_threshold2, max_mae, min_mae))


In [ ]:
indices=1
bar_width=1
for i in range (0,10):
    plt.bar(indices, confid[i], bar_width, label='Normal', color='blue', bottom=0)
    plt.ylim(0, 1)
    plt.show()

In [ ]:
autoencoder.save('/kaggle/working/autoencoder')

In [ ]:
import seaborn as sns
# Plot the histograms of the reconstruction losses
mae=1
mse=0
plt.figure(figsize=(15, 6))
if(mse):
  sns.distplot(mse_norm, bins=50, kde=True, label='normal')
  sns.distplot(mse_abnormal, bins=50, kde=True, label='abnormal')
elif(mae):
  sns.distplot(mae_norm, bins=50, kde=True, label='normal')
  sns.distplot(mae_abnormal, bins=50, kde=True, label='abnormal')
xlim=np.arange(0,0.03,0.001)
plt.xlim([0, 0.03])
# Add vertical line at x=0.01
plt.axvline(x=best_threshold2, color='r', linestyle='--', label='Threshold')
plt.xticks(xlim)
plt.xlabel('Reconstruction loss: MAE')
plt.ylabel('Frequency')
plt.legend(loc='upper right')
plt.show()